I'd share to share my cross validation work showing how model parameters influence cross-validation test performance. I define a few functions before showing some results. 

In [ ]:
import pandas, numpy
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime
from sklearn.metrics import r2_score

# Input data files are available in the "../input/" directory.
train = pandas.read_csv("../input/train.csv")
test = pandas.read_csv("../input/test.csv")

This function randomly splits the training data into training and validation sets. I define the arguments for the next function which serves as a wrapper.

In [ ]:
def regCheck(data, propTrain, models, features, outcome, regNames = None):
    ind = data.index.values
    size = int(numpy.round(len(ind)*propTrain))
    use = numpy.random.choice(ind, size, replace = False)
    train = data.loc[use]
    test = data.loc[set(ind) - set(use)]
    regmeas = []
    if regNames == None:
        names = []
    for m in models:
        if regNames == None:
            names.append(str(m).split("(")[0])
        trained = m.fit(train[features], train[outcome])
        test["prediction"] = trained.predict(test[features])
        out = r2_score(test[outcome], test["prediction"])
        regmeas.append(out)
    regmeas = pandas.DataFrame(regmeas)
    regmeas = regmeas.transpose()
    if regNames == None:
        regmeas.columns = names
    else:
        regmeas.columns = regNames
    return(regmeas)

The function below is used to compare competing models.

The data argument is a data frame with the features and outcome.

nsamples is the number of replications of spliting the data into training and test.

propTrain is the proportion of cases assigned to the training set.

models is a list of sklearn regressors (even a single classifier needs to be in a list).

features is a list of predictor variables.

outcome is the continuous outcome of interest.

regNames allows the user to specific names for the models to display in the output.

maxTime is the maximum number of minutes the function should be allowed to run.

This returns a data frame summarizing how the models performed.


In [ ]:
def simmer(data, models, features, outcome, nsamples = 100, propTrain = .8, regNames = None, maxTime = 1440):
    tstart = datetime.now()
    sd = dict()
    for i in range(0, nsamples):
        sd[i] = regCheck(data, propTrain, models, features, outcome, regNames)
        if (datetime.now() - tstart).seconds/60 > maxTime:
            print("Stopped at " + str(i + 1) + " replications to keep things under " + str(maxTime) + " minutes")
            break
    output = pandas.concat(sd)
    output = output.reset_index(drop = True)
    return(output)

This is used to determine how changing a parameter of the model nfluences cross validation accuracy.

param indicates what parameter should be varied (e.g., alpha for Lasso).

model is a string indicating what model should be fitted (e.g., "Lasso" or "MLPRegressor").

values is a list which indicates what levels of the parameter indicated by param should be used.

the remaining arguments are passed to the simmer function defined above. Note that this function has different defaults.

In [ ]:
def paramTester(param, model, values, features, outcome, data, nsamples = 100, propTrain = .5, maxTime = 10):
    models = []
    names = []
    for v in values:
        models.append(eval(model + "(" + param + "=" + str(v) + ")"))
        names.append(param + " = " + str(v))
    out = simmer(data, models, features, outcome, nsamples, propTrain, regNames = names, maxTime = maxTime)
    return(out)

Now I need to do some quick data preparation.

In [ ]:
train = pandas.get_dummies(train, drop_first = True)
test = pandas.get_dummies(test, drop_first = True)

usevars = list(set(train.columns).intersection(test.columns))

usevars = numpy.sort(usevars)  

preds = usevars[1:]


And now I can use my functions to see how changing alpha with Lasso influences test performance on random validation sets. 

Here I am using 80% of the training data to train a model, and then the remaining 20% to gauge prediction accuracy with R-squared. I specify maxTime as .5 so that this will run in about 30 seconds (it can be more, never less).

In [ ]:
values = [.5, .25, .1, .01]
model = "Lasso"
param = "alpha"
        
output = paramTester("alpha", "Lasso", values, preds, "y", train, propTrain = .8, maxTime = .5)
output

Each row in output shows the results from a single split of the data. I am concerned about within row differences between the columns.  Because alpha = 0.01 column has the highest R-squared value in every row, I favor .01 over the other alpha values.

What if we look at other low values?

In [ ]:
values = [.05, .025, .01, .001]
model = "Lasso"
param = "alpha"
        
output = paramTester("alpha", "Lasso", values, preds, "y", train, propTrain = .8, maxTime = .5)
output

Based on the results shown above, it looks like an alpha in the .025 to .010 range should be preferred.

The code shown here provides an example of how to probe models and determine how to increase test model accuracy. 

This can be used to do a series of tests to influence the decision of what model should be used and what parameters should be specified.

The results presented here are simply an example, I am not saying that using Lasso with an alpha of .025 will get you a .590 on the leaderboard (around .550, maybe).

I hope this helps someone!